In [1]:
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler
import pandas as pd
import sys
from math import log2, floor
import dimod
import numpy as np
import os

# Generalized Knapsack Problem

Given N items, each item has a given weight and value/cost, the standard Knapsack Problem is to maximize the value by selecting a maximum of K items adding up to a maximum weight W.

In a generalized Knapsack problem, we are allowed to take multiple pieces for each item, so the variable can take discrete values 0,1,2,... up to max_num_of_pieces. Therefore we use Discrete Quadratic Model to tackle this optimization problem.

In [2]:
def build_knapsack_dqm(costs, weights, weight_capacity, max_num_of_pieces, verbose = False):
    """Construct DQM for the generalized knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        max_num_of_pieces(int):
            Maximum allowable pieces for each item
    Returns:
        Discrete quadratic model instance
    """
    max_num_of_pieces += 1 # also take into account the value 0
    pieces = range(max_num_of_pieces)
    
    # First guess the lagrange
    lagrange = max(costs)*0.2
    if verbose:
        print('lagrange:',lagrange)

    # Number of objects
    x_size = len(costs)

    # Lucas's algorithm introduces additional slack variables to
    # handle the inequality. M+1 binary slack variables are needed to
    # represent the sum using a set of powers of 2.
    M = floor(log2(weight_capacity))
    num_slack_variables = M + 1

    # Slack variable list for Lucas's algorithm. The last variable has
    # a special value because it terminates the sequence.
    y = [2**n for n in range(M)]
    y.append(weight_capacity + 1 - 2**M)
    
    dqm = DiscreteQuadraticModel()
    # Add variables
    for k in range(x_size):
        dqm.add_variable(max_num_of_pieces, label='x' + str(k))

    for k in range(num_slack_variables):
        dqm.add_variable(2, label='y' + str(k)) # either 0 or 1

    # Hamiltonian xi-xi terms
    for k in range(x_size):
        dqm.set_linear('x' + str(k), lagrange * (weights[k]**2) * (np.array(pieces) **2) - costs[k]*pieces)


    # # Hamiltonian xi-xj terms
    for i in range(x_size):
        for j in range(i + 1, x_size):
            biases_dict = {}
            for piece1 in pieces:
                for piece2 in pieces:
                    biases_dict[(piece1, piece2)]=(2 * lagrange * weights[i] * weights[j])*piece1*piece2

            dqm.set_quadratic('x' + str(i), 'x' + str(j), biases_dict)

    # Hamiltonian y-y terms
    for k in range(num_slack_variables):
        dqm.set_linear('y' + str(k), lagrange*np.array([0,1])* (y[k]**2))

    # Hamiltonian yi-yj terms 
    for i in range(num_slack_variables):
        for j in range(i + 1, num_slack_variables): 
            dqm.set_quadratic('y' + str(i), 'y' + str(j), {(1,1):2 * lagrange * y[i] * y[j]})

    # Hamiltonian x-y terms
    for i in range(x_size):
        for j in range(num_slack_variables):
            biases_dict = {}
            for piece1 in pieces:
                biases_dict[(piece1, 1)]=-2 * lagrange * weights[i] * y[j]*piece1

            dqm.set_quadratic('x' + str(i), 'y' + str(j), biases_dict) 
    
    return dqm

def solve_knapsack(costs, weights, weight_capacity, max_num_of_pieces, sampler = None, verbose = False):
    """Construct DQM and solve the generalized knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        max_num_of_pieces(int):
            Maximum allowable pieces for each item
        sampler (DQM sampler instance or None):
            A DQM sampler instance or None, in which case
            LeapHybridSampler is used by default        
    Returns:
        Tuple:
            List of indices of selected items
            List of number of pieces for each selected item
            Solution energy
    """
    
    dqm = build_knapsack_dqm(costs, weights, weight_capacity, max_num_of_pieces)
    
    if sampler is None:
        sampler = LeapHybridDQMSampler()
    sampleset = sampler.sample_dqm(dqm)
    sample = sampleset.first.sample
    energy = sampleset.first.energy

    if verbose:
        print("Solution: ", sample)
        print("Solution energy: ", energy)

    selected_item_indices = []
    selected_item_pieces = []
    for varname, value in sample.items():
        # For each "x" variable, check whether its value is set, which
        # indicates that the corresponding item is included in the
        # knapsack
        if value and varname.startswith('x'):
            # The index into the weight array is retrieved from the
            # variable name
            selected_item_indices.append(int(varname[1:]))
            selected_item_pieces.append(value)
    return selected_item_indices, selected_item_pieces, energy

** Execution on data **

In [3]:
# Load Data
data_file_name = os.path.join(os.getcwd(),'data/large.csv')
df = pd.read_csv(data_file_name, names=['cost', 'weight'])
df

,cost,weight
0,77,95
1,44,70
2,15,85
3,67,31
4,75,100
...,...,...
63,88,23
64,8,73
65,1,41
66,71,13


In [4]:
# parse input data
costs = df['cost']
weights = df['weight']
weight_capacity = 100
max_num_of_pieces = 3

selected_item_indices, selected_item_pieces, energy = solve_knapsack(costs, weights, weight_capacity, max_num_of_pieces)

selected_weights_each = list(df.loc[selected_item_indices,'weight'])
selected_weights = np.multiply(np.array(selected_item_pieces), np.array(selected_weights_each))
selected_costs_each = list(df.loc[selected_item_indices,'cost'])
selected_costs = np.multiply(np.array(selected_item_pieces), np.array(selected_costs_each))
print("Selected item numbers:", selected_item_indices)
print("Number of pieces:", selected_item_pieces)
print("Selected item weights: {}, total = {}".format(selected_weights, sum(selected_weights)))
print("Selected item costs: {}, total = {}".format(selected_costs, sum(selected_costs)))

Selected item numbers: [5, 66, 67]
Number of pieces: [3, 3, 1]
Selected item weights: [ 3 39 44], total = 86
Selected item costs: [273 213  36], total = 522


# Application in Stock Portofolio Optimization

The above generalized Knapsack problem has direct application in stock portofolio optimization. Here we give a proof-of-concept demonstration.

Problem description: Say we currently have cash $10,000 and we want to invest in different stocks. Assume that for each stock, we can only buy at most max_num_of_shares_allowed shares and we want to maximize the profits quantified by earnings. These assumptions and profit quantification are not ideal for real investment, but can serve as a quick demonstration given the time constrainst in this hackathon.

Stock data is downloaded from here: https://datahub.io/core/s-and-p-500-companies-financials#data

Data description: List of companies in the S&P 500 (Standard and Poor’s 500). The S&P 500 is a free-float, capitalization-weighted index of the top 500 publicly listed stocks in the US (top 500 by market cap). The dataset includes a list of all the stocks contained therein and associated key financials such as price, market capitalization, earnings, price/earnings ratio, price to book etc.

In [8]:
# Load data
data_file_name = os.path.join(os.getcwd(),'data/constituents-financials_csv.csv')
df = pd.read_csv(data_file_name)

price_per_share = df['Price']
earnings_per_share = df['Earnings/Share']
cash = 10000 
max_num_of_shares_allowed = 4

df.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


** Execute the Knapsack DQM model on the real stock data **

In [9]:
selected_item_indices, selected_item_pieces, energy = solve_knapsack(
    costs = earnings_per_share, 
    weights = price_per_share, 
    weight_capacity = cash, 
    max_num_of_pieces = max_num_of_shares_allowed)

selected_symbol = list(df.loc[selected_item_indices,'Symbol'])
selected_name = list(df.loc[selected_item_indices,'Name'])

selected_prices_each = list(df.loc[selected_item_indices,'Price'])
selected_prices = np.multiply(np.array(selected_item_pieces), np.array(selected_prices_each))

selected_earnings_each = list(df.loc[selected_item_indices,'Earnings/Share'])
selected_earnings = np.multiply(np.array(selected_item_pieces), np.array(selected_earnings_each))

print("Selected stock:", selected_name)
print("\n")
print("Selected stock shares:", selected_item_pieces)
print("\n")
print("Selected stock costs: {}, total = {}".format(selected_prices, sum(selected_prices)))
print("\n")
print("Selected stock earnings: {}, total = {}".format(selected_earnings, sum(selected_earnings)))
print("\n")
print("Earning percentage for this portofolio: Earning / Investment Capital = {:0.2}%".format(100*sum(selected_earnings)/sum(selected_prices)) )

Selected stock: ['The Walt Disney Company', 'Tiffany & Co.', 'Time Warner Inc.', 'TJX Companies Inc.', 'TransDigm Group', 'Twenty-First Century Fox Class A', 'Twenty-First Century Fox Class B', 'Tyson Foods', 'UDR Inc', 'Ulta Salon Cosmetics & Fragrance Inc', 'Under Armour Class A', 'Under Armour Class C', 'Union Pacific', 'United Health Group Inc.', 'United Parcel Service', 'United Rentals, Inc.', 'Universal Health Services, Inc.', 'Unum Group', 'Valero Energy', 'Ventas Inc', 'Verisign Inc.', 'Verizon Communications', 'Visa Inc.', 'Vornado Realty Trust', 'Vulcan Materials', 'Wal-Mart Stores', 'Walgreens Boots Alliance', 'Waste Management Inc.', 'Wells Fargo', 'Welltower Inc.', 'Western Digital', 'Weyerhaeuser Corp.', 'Williams Cos.', 'Willis Towers Watson', 'Wyndham Worldwide', 'Wynn Resorts Ltd', 'Xilinx Inc', 'XL Capital', 'Xylem Inc.', 'Yum! Brands Inc', 'Zimmer Biomet Holdings', 'Zions Bancorp']


Selected stock shares: [1, 3, 1, 4, 2, 1, 2, 2, 4, 1, 3, 3, 3, 1, 2, 3, 2, 2, 4, 3, 